# Goal: Forecast of the planned flight using weather data 2 hours before expected departure
We need to get the data of the weather on the airport for that takeof time minus 2 hours.

steps:
1. Getting the flights.csv
2. making a loop that uses the time and -1 with the API to add the data to the given variable names.
3. Merges the datasets.
4. Save the CSV

In [33]:
import pandas as pd
import os

os.chdir('C:/Users/megar/OneDrive/Documenten/Business Analytics Management/Exchange/Course/Predictive Analytics/Group assignment/DBA3803/Data')


df = pd.read_csv('flights_withcode.csv')

# summary description
print(df.describe())


            YEAR         MONTH           DAY   DAY_OF_WEEK  FLIGHT_NUMBER  \
count  5819079.0  5.819079e+06  5.819079e+06  5.819079e+06   5.819079e+06   
mean      2015.0  6.524085e+00  1.570459e+01  3.926941e+00   2.173093e+03   
std          0.0  3.405137e+00  8.783425e+00  1.988845e+00   1.757064e+03   
min       2015.0  1.000000e+00  1.000000e+00  1.000000e+00   1.000000e+00   
25%       2015.0  4.000000e+00  8.000000e+00  2.000000e+00   7.300000e+02   
50%       2015.0  7.000000e+00  1.600000e+01  4.000000e+00   1.690000e+03   
75%       2015.0  9.000000e+00  2.300000e+01  6.000000e+00   3.230000e+03   
max       2015.0  1.200000e+01  3.100000e+01  7.000000e+00   9.855000e+03   

       SCHEDULED_DEPARTURE  DEPARTURE_TIME  DEPARTURE_DELAY      TAXI_OUT  \
count         5.819079e+06    5.732926e+06     5.732926e+06  5.730032e+06   
mean          1.329602e+03    1.335204e+03     9.370158e+00  1.607166e+01   
std           4.837518e+02    4.964233e+02     3.708094e+01  8.895574e+00  

In [34]:
# print the columns
print(df.columns)

# print the amount of NA values for origin airport

print(df['ORIGIN_AIRPORT'].isna().sum())

# print a summary of the data originairports
print(df['ORIGIN_AIRPORT'].describe())

# What are the attributes of the entries of origin airport that are not string type?
print(df['ORIGIN_AIRPORT'].apply(type).value_counts())

# give me some information on the int type entries of origin airport, like summary of what is most common
print(df['ORIGIN_AIRPORT'].apply(type).value_counts())

# print only the integers in the origin airport column and dont do value count
print(df['ORIGIN_AIRPORT'].apply(type))

# give all distinct values of the origin airport column
print(df['ORIGIN_AIRPORT'].unique())

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')
0
count     5819079
unique        322
top           ATL
freq       379424
Name: ORIGIN_AIRPORT, dtype: object
<class 'str'>    5819079
Name: ORIGIN_AIRPORT, dtype: int64
<class 'str'>    5819079
Name: ORIGIN_AIRPORT, dtype: int64
0          <class 'str'>
1          <class 'str'>
2          <class 'str'>
3          <class 'str'>
4          <class 'str'>
               ...      
5819074    <class 'str'>
5819075    <class 'str'>
581

In [35]:
# add a column that give the ICAO code for the origin airport, # add a column that give the ICAO code for the origin airport it is only a K added in front of the ITAO origin airport code
df['ORIGIN_AIRPORT_ICAO'] = 'K' + df['ORIGIN_AIRPORT'].str.strip()

# give summary statistics for the dataset
df.describe()

# see how many na are there in ORIGIN_AIRPORT_ICAO
print(df['ORIGIN_AIRPORT_ICAO'].isna().sum())



0


  ORIGIN_AIRPORT ORIGIN_AIRPORT_ICAO
0            ANC                KANC
1            LAX                KLAX
2            SFO                KSFO
3            LAX                KLAX
4            SEA                KSEA
624
KATL      346836
KORD      285884
KDFW      239551
KDEN      196055
KLAX      194673
           ...  
K14222         1
K14025         1
K13502         1
K15497         1
K12265         1
Name: ORIGIN_AIRPORT_ICAO, Length: 624, dtype: int64


In [38]:
import pandas as pd
from datetime import timedelta
import requests
import time
import re


# Function to parse scheduled departure time from HHMM format to timedelta
def parse_scheduled_departure(departure_time):
    departure_time_str = str(departure_time).zfill(4)
    hours = int(departure_time_str[:2])
    minutes = int(departure_time_str[2:])
    return timedelta(hours=hours, minutes=minutes)

# Function to fetch weather data
def fetch_weather(icao_code, year, month, day, departure_time):
    # Calculate the weather request time (departure time minus 2 hours)
    scheduled_delta = parse_scheduled_departure(departure_time)
    departure_datetime = pd.Timestamp(year=year, month=month, day=day) + scheduled_delta
    weather_request_time = departure_datetime - timedelta(hours=2)
    
    # Format the datetime for the API call
    date_str = weather_request_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Construct the API URL
    url = f"https://aviationweather.gov/api/data/metar?ids={icao_code}&format=raw&date={date_str}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        # Assuming the response is a plain text with METAR information
        data = response.text  # Use response.text as it's not JSON
        return data
    except Exception as e:
        print(f"Error fetching data for {icao_code} at {date_str}: {e}")
        return None

# Function to parse METAR data
def parse_metar(metar_string):
    if not metar_string:
        return pd.Series([None]*6, index=['WIND', 'VISIBILITY', 'SKY_CONDITION', 'TEMPERATURE', 'DEW_POINT', 'PRESSURE'])
    
    try:
        # Parse wind (00000KT)
        wind_match = re.search(r'(\d{5})KT', metar_string)
        wind = wind_match.group(1) if wind_match else None

        # Parse visibility (10SM)
        visibility_match = re.search(r'(\d+)SM', metar_string)
        visibility = visibility_match.group(1) if visibility_match else None

        # Parse sky condition (CLR, SCT, BKN, etc.)
        sky_condition_match = re.search(r'\b(CLR|FEW|SCT|BKN|OVC)\b', metar_string)
        sky_condition = sky_condition_match.group(1) if sky_condition_match else None

        # Parse temperature and dew point (18/13)
        temp_dew_match = re.search(r'(\d{2})/(\d{2})', metar_string)
        temperature = temp_dew_match.group(1) if temp_dew_match else None
        dew_point = temp_dew_match.group(2) if temp_dew_match else None

        # Parse pressure (A3001)
        pressure_match = re.search(r'A(\d{4})', metar_string)
        pressure = pressure_match.group(1) if pressure_match else None

        return pd.Series([wind, visibility, sky_condition, temperature, dew_point, pressure],
                         index=['WIND', 'VISIBILITY', 'SKY_CONDITION', 'TEMPERATURE', 'DEW_POINT', 'PRESSURE'])
    except Exception as e:
        print(f"Error parsing METAR string: {metar_string} - {e}")
        return pd.Series([None]*6, index=['WIND', 'VISIBILITY', 'SKY_CONDITION', 'TEMPERATURE', 'DEW_POINT', 'PRESSURE'])

# Fetch weather data and store parsed results in a new dataframe
weather_data_list = []

# Limit the dataframe to 100 rows
df_sample = df.head(100)

# Iterate through each row and collect weather data
for index, row in df_sample.iterrows():
    icao_code = row['ORIGIN_AIRPORT_ICAO']
    year = 2015  # As you mentioned, the year is always 2015
    month = row['MONTH']
    day = row['DAY']
    departure_time = row['SCHEDULED_DEPARTURE']
    
    # Fetch the weather data
    metar_string = fetch_weather(icao_code, year, month, day, departure_time)
    weather_data = parse_metar(metar_string)
    weather_data_list.append(weather_data)
    
    # Optional: Delay to respect API rate limits
    time.sleep(1)
    
    # For demonstration, limit the number of rows processed
    if index >= 99:
        break

# Add the fetched weather data as new columns
weather_df = pd.DataFrame(weather_data_list)
df_sample = pd.concat([df_sample, weather_df], axis=1)

print(df_sample.head())



   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  ...  AIRLINE_DELAY  \
0            ANC                 SEA                    5  ...            NaN   
1            LAX                 PBI                   10  ...            NaN   
2            SFO                 CLT                   20  ...            NaN   
3            LAX                 MIA                   20  ...            NaN   
4            SEA                 ANC                   25  ...            NaN   

   LATE_AIRCRAFT_DELAY  WEATHER_DELAY  ORIGIN_AIRPORT_ICAO   WIND  VISIBILITY  \
0    